In [56]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
path = f"{getcwd()}/data/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [58]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>.999):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [59]:
train_datagen = ImageDataGenerator(rescale=1/255)

In [60]:
train_generator = train_datagen.flow_from_directory( '/tmp/h-or-s/', target_size=(150, 150), batch_size=10, class_mode='binary')

Found 80 images belonging to 2 classes.


In [61]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

In [62]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 18496)            

In [63]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

In [64]:
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1, callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 1s 112ms/step - loss: 2.8931 - accuracy: 0.4250
Epoch 2/15
8/8 [==============================] - 1s 113ms/step - loss: 0.6477 - accuracy: 0.5125
Epoch 3/15
8/8 [==============================] - 1s 113ms/step - loss: 0.5315 - accuracy: 0.6875
Epoch 4/15
8/8 [==============================] - 1s 113ms/step - loss: 0.4812 - accuracy: 0.7750
Epoch 5/15
8/8 [==============================] - 1s 113ms/step - loss: 0.1612 - accuracy: 0.9750
Epoch 6/15
8/8 [==============================] - 1s 113ms/step - loss: 0.0487 - accuracy: 0.9875
Epoch 7/15
8/8 [==============================] - ETA: 0s - loss: 0.0314 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 1s 113ms/step - loss: 0.0314 - accuracy: 1.0000
